# Исследование надёжности заёмщиков

Посталвенная задача: выяснить, как влияет семейное положение и количество детей на факт погашения кредита в срок и влияет ли вообще.



Импортируем библиотеки и посмотрим на данные

In [ ]:
import pandas as pd

data.info()

print('возраст', data['dob_years'].unique()) # возраст 0
print('образование', data['education'].unique()) 
print('семейное положение', data['family_status'].unique()) 
print('вид дохода', data['income_type'].unique())
print('цель', data['purpose'].unique())
print('долг', data['debt'].unique()) 
data['education_lowercase'] = data['education'].str.lower()
print('образование', data['education_lowercase'].unique())
print('семейное положение', data['family_status_id'].unique())
print('пол', data['gender'].unique())
print(data['children'].value_counts()) #76 человек с 20 детьми, явно какая-то ошибка или опечатка
print(data['gender'].value_counts())
data.loc[data['gender'] == 'XNA', 'gender'] = 'F' 
data.loc[data['children'] == 20, 'children'] = 2 
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts())
print(data['gender'].value_counts())
print('пол', data['gender'].unique())
print(data['dob_years'].mean())
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()
print('пол', data['gender'].unique())

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
возраст [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
образование ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное в

методом unique() хочу проверить, какие значения есть в столбцах; интересные значения в детях 20, -1. Опечатка? в столбце возраст встречается значение 0, заполним его средним

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
print(data.isna().sum())
data['days_employed'] = abs(data['days_employed'])



total_income_mean = data.groupby('income_type')['days_employed'].mean() 

# для каждого типа занятости из серии, составленной выше
for value in total_income_mean.index:
    # в нашем датафрейме мы выбираем только те строки, где тип занятости соответствует выбранному выше (по очереди), и берем из стаж
    # и приравниваем их к самим себе (сохраняем), только заполняя пропуски доходом из таблички со средним доходом
    data.loc[data['income_type'] == value,'total_income'] = data.loc[data['income_type'] == value,'total_income'].fillna(total_income_mean.loc[value]) 

days_employed_mean = data.groupby('income_type')['days_employed'].mean()
for value in days_employed_mean.index:
    data.loc[data['income_type'] == value, 'days_employed'] = data.loc[data['income_type'] == value, 'days_employed'].fillna(days_employed_mean.loc[value])
print(data.isna().sum())

children                  0
days_employed          2174
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income           2174
purpose                   0
education_lowercase       0
dtype: int64
children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
education_lowercase    0
dtype: int64


**Вывод**

Заменила отсутствующие данные медианой

### Замена типа данных

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children               21525 non-null int64
days_employed          21525 non-null int64
dob_years              21525 non-null int64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null int64
purpose                21525 non-null object
education_lowercase    21525 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


изменили тип данных days_employed и dob_years на целочисленные; так как тип является типом float, использую метод astype(int)

### Обработка дубликатов

In [ ]:
print(data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
data.info()


54
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 13 columns):
children               21471 non-null int64
days_employed          21471 non-null int64
dob_years              21471 non-null int64
education              21471 non-null object
education_id           21471 non-null int64
family_status          21471 non-null object
family_status_id       21471 non-null int64
gender                 21471 non-null object
income_type            21471 non-null object
debt                   21471 non-null int64
total_income           21471 non-null int64
purpose                21471 non-null object
education_lowercase    21471 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


**Вывод**

изменилось общее количество при удалении дубликатов

### Лемматизация

In [ ]:
from pymystem3 import Mystem
m = Mystem()
data['lemmas'] = data['purpose'].apply(lambda text: m.lemmatize(text))
print(data['lemmas'].value_counts())
#лемматизируем цели
from collections import Counter
Counter(data['purpose'].apply(m.lemmatize).sum())
print(Counter(data['purpose'])) 

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

совершенно очевидно выделяются цели: образование, недвижимость, автомобиль, свадьба

### Категоризация данных

In [ ]:

 
def purposes_categories(purpose): # категоризация по цели
    if 'недвижим' in purpose or 'жил' in purpose:
            return 'Недвижимость'
    elif 'автомобил' in purpose:
            return 'Автомобиль'
    elif 'образован' in purpose:
            return 'Образование'
    elif 'свадьб' in purpose:
            return 'Свадьба'
 
data['purposes_categories'] = data['purpose'].apply(purposes_categories)
 
print(data['purposes_categories'].unique()) #смотрим значения в столбце

#создадим новую таблицу
data_category = data[['purposes_categories', 'debt', 'children', 'family_status_id']] 



['Недвижимость' 'Автомобиль' 'Образование' 'Свадьба']


**Вывод**

выделили 4 цели кредита

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_category_child_cor = data[['debt', 'children', 'dob_years']]
data_pivot = data_category_child_cor.pivot_table(index=['debt'], columns='children', aggfunc='count')
print(data_pivot)

data_category_children = data_category[data_category['children'] != 0]
data_category_children = data_category_children[data_category_children['debt'] == 1]
print('доля долга детных', data_category_children.count()/data_category.count())

data_category_no_children = data_category[data_category['children'] == 0]
data_category_no_children = data_category_no_children[data_category_no_children['debt'] == 1]

print('доля долга бездетных', data_category_no_children.count()/data_category.count())

         dob_years                                  
children         0       1       2      3     4    5
debt                                                
0          13044.0  4411.0  1926.0  303.0  37.0  9.0
1           1063.0   445.0   202.0   27.0   4.0  NaN
доля долга детных purposes_categories    0.031577
debt                   0.031577
children               0.031577
family_status_id       0.031577
dtype: float64
доля долга бездетных purposes_categories    0.049509
debt                   0.049509
children               0.049509
family_status_id       0.049509
dtype: float64


Доля бездетных с долгом больше, чем доля детных с долгом

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_category_family_status = data[['debt', 'family_status_id', 'total_income']]
data_pivot = data_category_family_status.pivot_table(index=['debt'], columns='family_status_id', aggfunc='count')
print(data_pivot)

data_category_married = data_category[data_category['family_status_id'] == 0]
data_category_married = data_category_married[data_category_married['debt'] == 1]

data_category_civil = data_category[data_category['family_status_id'] == 1]
data_category_civil = data_category_civil[data_category_civil['debt'] == 1]

data_category_widow = data_category[data_category['family_status_id'] == 2]
data_category_widow = data_category_widow[data_category_widow['debt'] == 1]

data_category_divorced = data_category[data_category['family_status_id'] == 3]
data_category_divorced = data_category_divorced[data_category_divorced['debt'] == 1]

data_category_single = data_category[data_category['family_status_id'] == 4]
data_category_single = data_category_single[data_category_single['debt'] == 1]

data_category_debt = data_category[data_category['debt'] == 1]

print('доля долга замужних/женатых', data_category_married.count()/data_category.count())
print('доля долга живущих гражданским браком', data_category_civil.count()/data_category.count())
print('доля долга вдовствующих', data_category_widow.count()/data_category.count())
print('доля долга разведенных', data_category_divorced.count()/data_category.count())
print('доля долга одиноких', data_category_single.count()/data_category.count())

                 total_income                       
family_status_id            0     1    2     3     4
debt                                                
0                       11413  3775  896  1110  2536
1                         931   388   63    85   274
доля долга замужних/женатых purposes_categories    0.043361
debt                   0.043361
children               0.043361
family_status_id       0.043361
dtype: float64
доля долга живущих гражданским браком purposes_categories    0.018071
debt                   0.018071
children               0.018071
family_status_id       0.018071
dtype: float64
доля долга вдовствующих purposes_categories    0.002934
debt                   0.002934
children               0.002934
family_status_id       0.002934
dtype: float64
доля долга разведенных purposes_categories    0.003959
debt                   0.003959
children               0.003959
family_status_id       0.003959
dtype: float64
доля долга одиноких purposes_categories    0.01276

**Вывод**

Большая доля с долгом - те, кто состоят в браке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
print(data['total_income'].max()) #пройдемся по значениям в столбце total_income
print(data['total_income'].min()) 
print(data['total_income'].median())

import numpy as np

data['total_income_cat_1']=pd.qcut(data.total_income, 4, labels=["низкий доход", "ниже среднего", "выше среднего", "высокий"])

print(data['total_income_cat_1'].value_counts())


data_category_income = data[['debt', 'total_income_cat_1', 'total_income']]
data_pivot = data_category_income.pivot_table(index=['debt'], columns='total_income_cat_1', aggfunc='count')
print(data_pivot)




2265604
520
138973.0
высокий          5368
ниже среднего    5368
низкий доход     5368
выше среднего    5367
Name: total_income_cat_1, dtype: int64
                   total_income                                    
total_income_cat_1 низкий доход ниже среднего выше среднего высокий
debt                                                               
0                          4944          4916          4886    4984
1                           424           452           481     384


<div class="alert alert-info">
сейчас получается примерно одинаковое количество человек всего и человек с долгом в каждой категории </div>



- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_category_real_estate = data[['debt', 'purposes_categories', 'total_income']]
data_pivot = data_category_real_estate.pivot_table(index=['debt'], columns='purposes_categories', aggfunc='count')
print(data_pivot)

data_category_real_estate = data_category[data_category['purposes_categories'] == 'Недвижимость']
data_category_real_estate = data_category_real_estate[data_category_real_estate['debt'] == 1]
print('доля с долгом за недвижимость', data_category_real_estate.count()/data_category.count())

data_category_car = data_category[data_category['purposes_categories'] == 'Автомобиль']
data_category_car = data_category_car[data_category_car['debt'] == 1]
print('доля с долгом за авто', data_category_car.count()/data_category.count())

data_category_education = data_category[data_category['purposes_categories'] == 'Образование']
data_category_education = data_category_education[data_category_education['debt'] == 1]
print('доля с долгом за образование', data_category_education.count()/data_category.count())

data_category_wedding = data_category[data_category['purposes_categories'] == 'Свадьба']
data_category_wedding = data_category_wedding[data_category_wedding['debt'] == 1]
print('доля с долгом за свадьбу', data_category_wedding.count()/data_category.count())

                    total_income                                 
purposes_categories   Автомобиль Недвижимость Образование Свадьба
debt                                                             
0                           3905        10032        3644    2149
1                            403          782         370     186
доля с долгом за недвижимость purposes_categories    0.036421
debt                   0.036421
children               0.036421
family_status_id       0.036421
dtype: float64
доля с долгом за авто purposes_categories    0.01877
debt                   0.01877
children               0.01877
family_status_id       0.01877
dtype: float64
доля с долгом за образование purposes_categories    0.017233
debt                   0.017233
children               0.017233
family_status_id       0.017233
dtype: float64
доля с долгом за свадьбу purposes_categories    0.008663
debt                   0.008663
children               0.008663
family_status_id       0.008663
dtype: floa

Большая доля с долгами - у тех, кто брал кредит на недвижимость; меньшая доля - у тех, кто брал кредит на свадьбу

In [ ]:
print(data['debt'].value_counts())
data_category_final_purpose = data[['debt', 'purposes_categories', 'total_income']]
data_pivot_purpose = data_category_final_purpose.pivot_table(index=['purposes_categories'], columns='debt', aggfunc='count')
print(data_pivot_purpose)

data_category_final_children = data[['debt', 'children', 'total_income']]
data_pivot_children = data_category_final_children.pivot_table(index=['children'], columns='debt', aggfunc='count')
print(data_pivot_children)

data_category_final_family = data[['debt', 'family_status', 'total_income']]
data_pivot_family = data_category_final_family.pivot_table(index=['family_status'], columns='debt', aggfunc='count')
print(data_pivot_family)

data_category_income = data[['debt', 'total_income_cat_1', 'total_income']]
data_pivot = data_category_income.pivot_table(index=['debt'], columns='total_income_cat_1', aggfunc='count')
print(data_pivot)

data_pivot_purpose = data.pivot_table(index = 'purposes_categories', values = 'debt', aggfunc = ['sum','count'])
data_pivot_purpose.columns = ['debt_number', 'total_number']
data_pivot_purpose['percent'] = data_pivot_purpose['debt_number'] / data_pivot_purpose['total_number'] * 100
print(data_pivot_purpose)

data_pivot_children = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['sum','count'])
data_pivot_children.columns = ['debt_number', 'total_number']
data_pivot_children['percent'] = data_pivot_children['debt_number'] / data_pivot_children['total_number'] * 100
print(data_pivot_children)

data_pivot_family = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['sum','count'])
data_pivot_family.columns = ['debt_number', 'total_number']
data_pivot_family['percent'] = data_pivot_family['debt_number'] / data_pivot_family['total_number'] * 100
print(data_pivot_family)

data_pivot_income = data.pivot_table(index = 'total_income_cat_1', values = 'debt', aggfunc = ['sum','count'])
data_pivot_income.columns = ['debt_number', 'total_number']
data_pivot_income['percent'] = data_pivot_income['debt_number'] / data_pivot_income['total_number'] * 100
print(data_pivot_income)

0    19730
1     1741
Name: debt, dtype: int64
                    total_income     
debt                           0    1
purposes_categories                  
Автомобиль                  3905  403
Недвижимость               10032  782
Образование                 3644  370
Свадьба                     2149  186
         total_income        
debt                0       1
children                     
0             13044.0  1063.0
1              4411.0   445.0
2              1926.0   202.0
3               303.0    27.0
4                37.0     4.0
5                 9.0     NaN
                      total_income     
debt                             0    1
family_status                          
Не женат / не замужем         2536  274
в разводе                     1110   85
вдовец / вдова                 896   63
гражданский брак              3775  388
женат / замужем              11413  931
                   total_income                                    
total_income_cat_1 низкий дох

## Шаг 4. Общий вывод

Общее количество человек с долгом составляет 1741.
Итого: имеют задолженность по кредиту: те, кто брал на недвижимость; у кого доход больше медианного; замужем/женат; у кого нет детей.
Количество детей согласно данным имеет следующую зависимость: чем больше детей, тем меньше вероятность задолжать по кредиту, однако, скорее всего в этом случае ужесточаются требования банков, поэтому людис большим количеством детей просто-напросто не получают кредиты, а те, кто получают, имеют хорошие финансовые возможности.